In [10]:
include("framework.jl")
include("create.jl")
configPath="../data/config";

In [11]:
config = initConfig();
seed = 0;  Random.seed!(seed);

addIco(config, 0, 0.0, 0.0, 0.0, 9, 2.0, 1.0, 1.0)

for i in 1:100
    randomIcoSphere( 10, config, 1, 3, 0.1 )
end

write_csv(config, configPath*"/test.csv")

"../data/config/test.csv"